In [4]:
import numpy as np
import os
import pandas as pd

#
#
#
#
# PER ORA E' PRESO DAI FILE LOCALI, VEDI SE C'E' UN ALTRO MODO
#
#
#
#

os.chdir('/Users/lilit/Downloads')

quasar_data = pd.read_csv('galaxyquasar.csv')
usecols = ['u', 'g', 'r', 'i', 'z', 'class', 'z1', 'zerr']
# USECOLS E' IL NOME, GET E' IL METODO
print(quasar_data)


# SALVA TUTTO IN NP.ARRAY!

#q_class = np.array(quasar_data.get(usecols[1]))

u_g = np.array(quasar_data.get(usecols[0]) - quasar_data.get(usecols[1]))
g_r = np.array(quasar_data.get(usecols[1]) - quasar_data.get(usecols[2]))
r_i = np.array(quasar_data.get(usecols[2]) - quasar_data.get(usecols[3]))
i_z = np.array(quasar_data.get(usecols[3]) - quasar_data.get(usecols[4]))

target = np.array(quasar_data.get(usecols[5])
class_label = np.empty(shape = size(u_g))
for i, label in enumerate(target):
    if (label == 'GALAXY'): class_label[i] = 0
    elif (label == 'QSO'): class_label[i] = 1


    
















              u         g         r         i         z   class        z1  \
0      18.97213  18.53676  18.58280  18.34936  18.29215     QSO  0.522819   
1      19.24592  17.47646  16.47817  16.04472  15.68851  GALAXY  0.122846   
2      19.43536  17.70268  16.91565  16.58327  16.39128  GALAXY  0.000000   
3      19.31626  18.18312  17.39591  16.94549  16.65395  GALAXY  0.147435   
4      19.28828  19.11188  18.88937  18.80013  18.49183     QSO  2.011455   
...         ...       ...       ...       ...       ...     ...       ...   
49995  19.37295  18.12382  17.39886  16.98503  16.70585  GALAXY  0.113016   
49996  18.52021  16.88262  16.03280  15.56884  15.22454  GALAXY  0.085063   
49997  18.62718  17.30876  16.87371  16.62399  16.42296  GALAXY  0.054429   
49998  19.55140  18.27711  17.62101  17.21947  17.03347  GALAXY  0.112571   
49999  18.80772  17.75751  17.40500  17.21650  17.12295  GALAXY  0.043652   

           zerr  
0      0.000155  
1      0.000028  
2      0.000000  
3  